# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [55]:
# Do all imports and installs here
import pandas as pd

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

This project is to integrate the three data source, and build the star schema.

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [ ]:
fname = '../../data2/GlobalLandTemperaturesByCity.csv'
df = pd.read_csv(fname)

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
enableHiveSupport().getOrCreate()

df_spark_temp = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


In [23]:
#write to parquet
# df_spark.write.parquet("sas_data")
df_spark_temp=spark.read.parquet("sas_data")

In [24]:
df_spark_temp.head()

Row(cicid=5748517.0, i94yr=2016.0, i94mon=4.0, i94cit=245.0, i94res=438.0, i94port='LOS', arrdate=20574.0, i94mode=1.0, i94addr='CA', depdate=20582.0, i94bir=40.0, i94visa=1.0, count=1.0, dtadfile='20160430', visapost='SYD', occup=None, entdepa='G', entdepd='O', entdepu=None, matflag='M', biryear=1976.0, dtaddto='10292016', gender='F', insnum=None, airline='QF', admnum=94953870030.0, fltno='00011', visatype='B1')

In [5]:
df_spark_temp.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

In [25]:
from pyspark.sql.functions import col,isnan, when, count
df_spark_temp.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_spark_temp.columns]).show()

+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+------+-----+--------+
|cicid|i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|  occup|entdepa|entdepd|entdepu|matflag|biryear|dtaddto|gender| insnum|airline|admnum|fltno|visatype|
+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+------+-----+--------+
|    0|    0|     0|     0|     0|      0|      0|    239| 152592| 142457|   802|      0|    0|       1| 1881250|3088187|    238| 138429|3095921| 138429|    802|    477|414269|2982605|  83627|     0|19549|       0|
+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-------+-------+-------+---

In [ ]:
df_spark_temp.columns

In [26]:
from datetime import datetime
def convert_sas_to_date(sas_date):
    if sas_date is None:
        dt = None
    else:
        dt = datetime.fromordinal(datetime(1960, 1, 1).toordinal() + int(sas_date))
    return dt

In [27]:
convert_sas_to_date(24843.0)

datetime.datetime(2028, 1, 7, 0, 0)

In [30]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DateType
convert_SAS_date = udf(lambda x: convert_sas_to_date(x), DateType())

In [31]:
df_spark_temp = df_spark_temp.withColumn("arrival_date", convert_SAS_date("arrdate"))
df_spark_temp = df_spark_temp.withColumn("departure_date", convert_SAS_date("depdate"))

In [32]:
df_spark_temp.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

In [33]:
df_spark = df_spark_temp.select(['cicid', 'i94yr', 'i94mon', 'i94res', 'i94port', 'arrival_date', \
             'i94mode', 'departure_date', 'i94visa', 'entdepa', 'dtaddto', 'visatype'])
column_list = ['cicid', 'year', 'month', 'residency_country', 'cbp_code', \
                   'arrival_date', 'transportation_mode', 'departure_date', 'visa', \
                  'arrival_flag', 'expiry_date', 'visa_type']
df_spark = df_spark.toDF(*column_list)

In [34]:
df_spark.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- year: double (nullable = true)
 |-- month: double (nullable = true)
 |-- residency_country: double (nullable = true)
 |-- cbp_code: string (nullable = true)
 |-- arrival_date: date (nullable = true)
 |-- transportation_mode: double (nullable = true)
 |-- departure_date: date (nullable = true)
 |-- visa: double (nullable = true)
 |-- arrival_flag: string (nullable = true)
 |-- expiry_date: string (nullable = true)
 |-- visa_type: string (nullable = true)



In [ ]:
spark.sparkContext.getConf().getAll()

In [21]:
df_test = df_spark.drop('departure_date')
df_test.head()

Row(cicid=5748517.0, year=2016.0, month=4.0, residency_country=438.0, cbp_code='LOS', arrival_date=datetime.date(2016, 4, 30), transportation_mode=1.0, visa=1.0, arrival_flag='G', expiry_date='10292016', visa_type='B1')

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [36]:
def code_mapper(idx):
    with open('I94_SAS_Labels_Descriptions.SAS', 'r') as f:
        f_content = f.read()
        f_content = f_content.replace('\t', '')

    f_content2 = f_content[f_content.index(idx):]
    f_content2 = f_content2[:f_content2.index(';')].split('\n')
    f_content2 = [i.replace("'", "") for i in f_content2]
    dic = [i.split('=') for i in f_content2[1:]]
    dic = dict([i[0].strip(), i[1].strip()] for i in dic if len(i) == 2)
    return dic

In [70]:
country_code_table = code_mapper('i94cntyl')
cbp_code_table = code_mapper('i94prtl')
transportation_mode_table = code_mapper('i94model')
state_code_table = code_mapper('i94model')
visa_table = code_mapper('visa_code')

In [75]:
def cbp_mapping(cbp_code):
    value = cbp_code_table.get(cbp_code)
    if ',' in value:
        return value.split(',')[0].strip(), value.split(',')[1].strip()
    else:
        return None, None

In [77]:
cbp_code_mapping = udf(lambda x: cbp_mapping(x))

In [81]:
df_spark.show()

+---------+------+-----+-----------------+--------+------------+-------------------+--------------+----+------------+-----------+---------+
|    cicid|  year|month|residency_country|cbp_code|arrival_date|transportation_mode|departure_date|visa|arrival_flag|expiry_date|visa_type|
+---------+------+-----+-----------------+--------+------------+-------------------+--------------+----+------------+-----------+---------+
|5748517.0|2016.0|  4.0|            438.0|     LOS|  2016-04-30|                1.0|    2016-05-08| 1.0|           G|   10292016|       B1|
|5748518.0|2016.0|  4.0|            438.0|     LOS|  2016-04-30|                1.0|    2016-05-17| 1.0|           G|   10292016|       B1|
|5748519.0|2016.0|  4.0|            438.0|     LOS|  2016-04-30|                1.0|    2016-05-08| 1.0|           G|   10292016|       B1|
|5748520.0|2016.0|  4.0|            438.0|     LOS|  2016-04-30|                1.0|    2016-05-14| 1.0|           G|   10292016|       B1|
|5748521.0|2016.0|  

In [80]:
df_spark = df_spark \
    .withColumn("port_city", cbp_code_mapping("cbp_code")[0]) \
    .withColumn("port_state", cbp_code_mapping("cbp_code")[1])

AnalysisException: "Can't extract value from <lambda>(cbp_code#672): need struct type but got string;"

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.